Telecom Churn Prediction - Прогнозирование оттока клиентов телекоммуникационной компании. Отток клиентов в нашем Dataset имеет скрытый характер - клиент продолжает пользоваться услугами компании, но объем и регулярность потребления снижается. Клиенты, которые пользуются услугами компании, при этом объем и регулярность потребления не снижается считаются лояльными к компании. Клиенты, у которых объем и регулярность потребления услуг компании снижается считаются нелояльными. Целевой переменной  (target) - является churn (отток клиентов). Основной задачей прогнозной модели является выявление и анализ основных факторов, связанных с величиной оттока или лояльностью клиента.

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

print(os.listdir("../input"))
sns.set(rc={'figure.figsize':(10, 8)});
df = pd.read_csv('../input/churn-in-telecoms-dataset/bigml_59c28831336c6604c800002a.csv')
df.head(10)






Для более полного ознакомления с Dataset просмотрим последние 10 строк. 

In [ ]:
df.tail(10)


Так как столбцов достаточно много транспонируем вывод для удобства изучения Dataset.

In [ ]:
df.head(10).T

Изучим общую информацию о датасете: тип каждого признака, наличие в данных пропусков.

In [ ]:
df.info()

Рассмотрим статистику по каждому числовому признаку.

In [ ]:
df.describe().T

Рассмотрим статистику по каждому нечисловому признаку.

In [ ]:
df.describe(include=['object','bool'])

Рассмотрим распределение абонентов по целевому признаку churn.

In [ ]:
df['churn'].value_counts(normalize=True)


Процент лояльных пользователей составляет 85,5086% 

In [ ]:

df['churn'].value_counts().plot(kind='pie',label='churn')
plt.legend()
plt.title('Распределение оттока клиентов');

Проанализируем, жители какого штата говорят больше всего? Меньше всего? 
 

Так как необходимо учитывать все виды звонков, то вначале найдем сумму длительности всех звонков:total day minutes, total eve minutes, total night minutes, total intl minutes.

In [ ]:
df['total minutes'] = df['total day minutes'] + df['total eve minutes'] + ['total night minutes'] + ['total intl minutes']
df.head()

Определим, верно ли что люди, совершающие много международных звонков, приобретают international plan?

In [ ]:
df.pivot_table(values=['total intl calls'],index= ['international plan'], aggfunc='mean')

In [ ]:
df.groupby('international plan')['total intl calls'].mean().plot(kind='bar') 
plt.ylabel('total intl calls') 
plt.show();

Приобретение international plan практически не связано с количеством международных звонков.

Проанализируем, как связаны наличие голосовой почты и международного плана с лояльностью абонента

In [ ]:
_, axes=plt.subplots(1,2,sharey=True,figsize=(16,6))
sns.countplot(x = 'international plan', hue = 'churn', data=df, ax=axes[0]);
sns.countplot(x = 'voice mail plan', hue = 'churn', data=df, ax=axes[1]);


Очевидно, что при наличии международного плана доля оттока намного выше. Пользование голосовой  почтой практически не влияет на отток клиентов.

Проанализируем, как связано количество обращений в сервисный центр и лояльность абонента.

In [ ]:
pd.crosstab(df['churn'],df['customer service calls'],margins=True)
sns.countplot(x='customer service calls', hue='churn',data=df)

Исходя из вышеприведенной диаграммы доля оттока возрастает, начиная с 4 звонков в сервисный центр.

Найдем числовой признак, имеющий наибольшую степень корреляции.

Определим, связано ли время пользования аккаунтом с количеством обращений в колл-центр? Для этого вычислим коэффициент корреляции 

In [ ]:
from scipy.stats import pearsonr, spearmanr, kendalltau
r = pearsonr(df['account length'], df['customer service calls'])
print('Pearson correlation:', r[0], 'p-value:', r[1])

Так как p-value > 0.05 (типичное пороговое значение), то делаем вывод о том, что взаимосвязь (корреляция) между временем пользования аккаунтом и количеством обращений в колл-центр статистически незначима.

# **Как можно еще проанализировать данную Dataset?**
1. Провести факторный анализ для выявления связанных величин
2. Построить boxplot-ы для описания статистики распределения количественных признаков в двух группах: среди лояльных и нелояльных клиентов
3. Определить связь длительности и количества звонков с целевой переменной
4. Определить территориальную зависимость для целевого признака
